In [3]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# Funktion zur Berechnung des RSI (falls ta-lib nicht verfügbar ist)
def calculate_rsi(data, periods=14):
 delta = data['Close'].diff()
 gain = (delta.where(delta > 0, 0)).rolling(window=periods).mean()
 loss = (-delta.where(delta < 0, 0)).rolling(window=periods).mean()
 rs = gain / loss
 rsi = 100 - (100 / (1 + rs))
 return rsi

# Funktion zur Identifikation von Elliott-Wellen (vereinfacht)
def identify_elliott_waves(data, window=20):
    # Vereinfachte Logik: Wir suchen nach Hoch- und Tiefpunkten, um Wellen zu identifizieren
    highs = data['High'].rolling(window =window).max()
    lows = data['Low'].rolling(window=window).min()
    
    # Markiere Hoch- und Tiefpunkte
    data['is_high'] = (data['High'] == highs)
    data['is_low'] = (data['Low'] == lows)
    
    # Identifiziere Wellenmuster (Impuls: 5 Wellen, Korrektur: 3 Wellen)
    wave_points = []
    for i in range(1, len(data)):
        if data['is_high'].iloc[i]:
            wave_points.append(('H', data.index[i], data['High'].iloc[i]))
        elif data['is_low'].iloc[i]:
            wave_points.append(('L', data.index[i], data['Low'].iloc[i]))
    
    return wave_points

# Funktion zur Berechnung von Fibonacci-Retracements
def calculate_fibonacci_levels(high, low):
    diff = high - low
    levels = {
    '23.6%': high - diff * 0.236,
    '38.2%': high - diff * 0.382,
    '50.0%': high - diff * 0.5,
    '61.8%': high - diff * 0.618,
    }
    return levels

# Hauptfunktion zur Kauf-/Verkaufsentscheidung
def trading_decision(data, wave_points):
    # Letzter Preis
    last_price = data['Close'].iloc [-1]
    
    # Berechne RSI
    data['RSI'] = calculate_rsi(data)
    last_rsi = data['RSI'].iloc[-1]
    
    # Identifiziere die letzten Wellen
    if len(wave_points) < 2:
        return "Nicht genug Daten für eine Analyse."
 
    # Letzte zwei Punkte für Fibonacci-Retracements
    last_two_points = wave_points[-2:]
    if last_two_points[0][0] == 'H' and last_two_points[1][0] == 'L':
        high = last_two_points[0][2]
        low = last_two_points[1][2]
    elif last_two_points[0][0] == 'L' and last_two_points[1][0] == 'H':
        low = last_two_points[0][2]
        high = last_two_points[1][2]
    else:
        return "Kein klares Wellenmuster erkannt."
    
    # Berechne Fibonacci-Levels
    fib_levels = calculate_fibonacci_levels(high, low)
    
    # Entscheidungslogik
    decision = "Halten"
    reasoning = []
    
    # Prüfe, ob der Preis in der Nähe eines Fibonacci-Levels ist
    for level, value in fib_levels.items():
        if abs(last_price - value) / value < 0.01: # Preis ist innerhalb von 1% des Levels
            reasoning.append(f"Preis in der Nähe des Fibonacci-Levels {level} ({value:.2f})")
        if last_rsi < 30: # RSI zeigt Überverkauft an
            decision = "Kaufen"
            reasoning.append("RSI zeigt Überverkauft an (< 30)")
        elif last_rsi > 70: # RSI zeigt Überkauft an
            decision = "Verkaufen"
            reasoning.append("RSI zeigt Überkauft an (> 70)")
    
    # Prüfe Wellenmuster (vereinfacht)
    if len(wave_points) >= 5:
        last_five = wave_points[-5:]
    # Prüfe, ob wir in einer Korrekturphase (Welle 4 oder A) sind
    if last_five[-1][0] == 'L' and last_five[-2][0] == 'H':
        reasoning.append("Möglicherweise in Korrekturphase (Welle 4 oder A)")
    if last_price <= fib_levels['61.8%'] and last_rsi < 40:
        decision = "Kaufen"
        reasoning.append("Günstiger Einstiegspunkt in Korrekturphase")
        
    return decision, reasoning

In [4]:

ticker = yf.Ticker("MSFT")


In [5]:
ticker.history()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2025-03-03 00:00:00-05:00,398.820007,398.820007,386.160004,388.489990,23007700,0.0,0.0
2025-03-04 00:00:00-05:00,383.399994,392.579987,381.000000,388.609985,29342900,0.0,0.0
2025-03-05 00:00:00-05:00,389.339996,401.670013,388.809998,401.019989,23433100,0.0,0.0
2025-03-06 00:00:00-05:00,394.279999,402.149994,392.679993,396.890015,23304600,0.0,0.0
2025-03-07 00:00:00-05:00,392.320007,394.799988,385.540009,393.309998,22034100,0.0,0.0
2025-03-10 00:00:00-04:00,385.839996,386.399994,377.220001,380.160004,32840100,0.0,0.0
2025-03-11 00:00:00-04:00,379.000000,386.000000,376.910004,380.450012,30380200,0.0,0.0
2025-03-12 00:00:00-04:00,382.950012,385.220001,378.950012,383.269989,24253600,0.0,0.0
2025-03-13 00:00:00-04:00,383.160004,385.320007,377.450012,378.769989,20473000,0.0,0.0


In [4]:
expirations = ticker.options

JSONDecodeError: Expecting value: line 1 column 1 (char 0)